In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/611.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/364.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/367.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/116.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/1490.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/374.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/1359.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/485.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/456.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/626.wav
/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream/590.wav
/kaggle/input/audio

In [2]:
import librosa
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import time
import os
import numpy as np
import joblib
#import sounddevice as sd
#from joblib import Parallel, delayed
#import gc  # For garbage collection to free memory

In [3]:
import torch
print(torch.cuda.is_available())

False


In [4]:
# Directories for Screaming and Not Screaming datasets
scream_dir = "/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/scream"
non_scream_dir = "/kaggle/input/audio-dataset-of-scream-and-non-scream/Converted_Separately/non_scream"
screaming_dir = "/kaggle/input/human-screaming-detection-dataset/Screaming"
not_screaming_dir = "/kaggle/input/human-screaming-detection-dataset/NotScreaming"

RATE = 44100  # Sample rate

In [5]:
def extract_single_audio_features(y):
    """Extracts various audio features including MFCCs, spectral contrast, chroma features, and more."""
    
    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=y, sr=RATE, n_mfcc=13)
    
    # Extract Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=RATE)
    
    # Extract Chroma Features
    chroma = librosa.feature.chroma_stft(y=y, sr=RATE)
    
    # Extract Zero Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    
    # Extract Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=RATE)
    
    # Extract Spectral Bandwidth
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=RATE)
    
    # Extract Root Mean Square Energy (RMSE)
    rmse = librosa.feature.rms(y=y)
    
    # Extract Spectral Roll-off
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=RATE)
    
    # Extract Tempo (Beats Per Minute)
    onset_env = librosa.onset.onset_strength(y=y, sr=RATE)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=RATE)
    
    # Concatenate all the features into a single feature vector
    feature_vector = np.hstack([
        np.mean(mfccs, axis=1),             # MFCCs
        np.mean(spectral_contrast, axis=1), # Spectral Contrast
        np.mean(chroma, axis=1),            # Chroma
        np.mean(zero_crossing_rate),        # Zero Crossing Rate
        np.mean(spectral_centroid),         # Spectral Centroid
        np.mean(spectral_bandwidth),        # Spectral Bandwidth
        np.mean(rmse),                      # RMSE
        np.mean(spectral_rolloff),          # Spectral Roll-off
        tempo                               # Tempo (BPM)
    ])
    
    return feature_vector

In [6]:
def load_and_process_audio_files(directory):
    """Load all audio files from a directory, process and extract features."""
    audio_features = []
    
    for root, _, files in os.walk(directory):
        for file in files[:20]:
            file_path = os.path.join(root, file)
            y, _ = librosa.load(file_path, sr=RATE)
            features = extract_single_audio_features(y)
            audio_features.append(features)
    
    return np.array(audio_features)

In [7]:
# Step 1: Load and process scream audio files
print("Loading and processing scream audio files...")
X_screams_1 = load_and_process_audio_files(screaming_dir)
print("1/2")
X_screams_2 = load_and_process_audio_files(scream_dir)
print("2/2")
    
# Combine scream datasets from both directories
X_screams = np.vstack([X_screams_1, X_screams_2])
print("Scream audio files processed and features extracted.")
    
# Step 2: Load and process non-scream audio files
print("Loading and processing non-scream audio files...")
X_non_screams_1 = load_and_process_audio_files(not_screaming_dir)
print("1/2")
X_non_screams_2 = load_and_process_audio_files(non_scream_dir)
print("2/2")
    
# Combine non-scream datasets from both directories
X_non_screams = np.vstack([X_non_screams_1, X_non_screams_2])
print("Non-scream audio files processed and features extracted.")
    
# Now you can proceed with model training using X_screams and X_non_screams
print(f"Scream features shape: {X_screams.shape}")
print(f"Non-scream features shape: {X_non_screams.shape}")


Loading and processing scream audio files...


/tmp/ipykernel_18/3316365667.py:30: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=RATE)


1/2
2/2
Scream audio files processed and features extracted.
Loading and processing non-scream audio files...
1/2
2/2
Non-scream audio files processed and features extracted.
Scream features shape: (40, 38)
Non-scream features shape: (40, 38)


In [8]:
# Step 3: Convert features to arrays and combine
X = np.vstack([X_screams, X_non_screams])
y_screams = np.ones(len(X_screams))
y_non_screams = np.zeros(len(X_non_screams))
y = np.hstack([y_screams, y_non_screams])
print("3. step completed...")

3. step completed...


In [9]:
print(X.shape)
print(y.shape)

(80, 38)
(80,)


In [10]:
# Step 4: Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)
print("4. step completed...")

4. step completed...


In [11]:
# Step 7: Handle class imbalance (optional)
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print("7. step completed...")

7. step completed...


In [12]:
# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("5. step completed...")

5. step completed...


In [13]:
# Step 6: Train classifiers with cross-validation
classifiers = [
    SVC(kernel='linear'), 
    LogisticRegression(max_iter=10000), 
    XGBClassifier(),
    LGBMClassifier(verbose = 0),
    CatBoostClassifier(verbose = False),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    MLPClassifier(max_iter=1000)
]

models = ["SVC", "LogisticRegression", "XGBClassifier", "LGBMClassifier","CatBoostClassifier", "RandomForestClassifier", "KNeighborsClassifier", "DecisionTreeClassifier", "MLPClassifier"]

i = 0
for classifier in classifiers:
    print(f"Training {models[i]}...")
    #scores = cross_val_score(classifier, X, y, cv=5)
    #print(f"Cross-Validation Accuracy: {np.mean(scores):.2f} (+/- {np.std(scores):.2f})")
    
    # Fit the model and make predictions
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    # Evaluation
    print(f"Accuracy:{round(accuracy_score(y_test, y_pred)*100)}")
    print(f"Confusion Matrix for {models[i]}:\n", confusion_matrix(y_test, y_pred))
    print(f"Classification Report for {models[i]}:\n", classification_report(y_test, y_pred))
    print("...................")
    i+=1

print("6. step completed...")

Training SVC...
Accuracy:88
Confusion Matrix for SVC:
 [[ 4  1]
 [ 1 10]]
Classification Report for SVC:
               precision    recall  f1-score   support

         0.0       0.80      0.80      0.80         5
         1.0       0.91      0.91      0.91        11

    accuracy                           0.88        16
   macro avg       0.85      0.85      0.85        16
weighted avg       0.88      0.88      0.88        16

...................
Training LogisticRegression...
Accuracy:94
Confusion Matrix for LogisticRegression:
 [[ 4  1]
 [ 0 11]]
Classification Report for LogisticRegression:
               precision    recall  f1-score   support

         0.0       1.00      0.80      0.89         5
         1.0       0.92      1.00      0.96        11

    accuracy                           0.94        16
   macro avg       0.96      0.90      0.92        16
weighted avg       0.94      0.94      0.94        16

...................
Training XGBClassifier...
Accuracy:62
Confusion M

In [14]:
print("RandomForestClassifier")

classifier = RandomForestClassifier()
    
# Fit the model and make predictions
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
    
# Evaluation
print(f"Accuracy :{round(accuracy_score(y_test, y_pred)*100, 2)}")
print(f"Confusion Matrix :\n", confusion_matrix(y_test, y_pred))
print(f"Classification Report :\n", classification_report(y_test, y_pred))
print("...................")

RandomForestClassifier
Accuracy :62.5
Confusion Matrix :
 [[4 1]
 [5 6]]
Classification Report :
               precision    recall  f1-score   support

         0.0       0.44      0.80      0.57         5
         1.0       0.86      0.55      0.67        11

    accuracy                           0.62        16
   macro avg       0.65      0.67      0.62        16
weighted avg       0.73      0.62      0.64        16

...................


In [15]:
# Save the trained model
joblib.dump(classifier, '/kaggle/working/scream_detector_model.pkl')
joblib.dump(scaler, '/kaggle/working/scaler.pkl' )

['/kaggle/working/scaler.pkl']

 ***REAL TIME SCREAM DETECTION***

In [16]:
# Constants
RATE = 44100  # Sample rate
CHUNK_DURATION = 1.0  # Duration of each chunk in seconds
CHUNK_SIZE = int(RATE * CHUNK_DURATION)  # Number of samples per chunk

In [17]:
# Load the pre-trained model and scaler
model = joblib.load('/kaggle/working/scream_detector_model.pkl')  # Replace with your trained model
scaler = joblib.load('/kaggle/working/scaler.pkl')  # Replace with the scaler used during training

In [18]:
# Define feature extraction function (same as used during training)
def extract_single_audio_features(y):
    """Extract features used in model training."""
    mfccs = librosa.feature.mfcc(y=y, sr=RATE, n_mfcc=13)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=RATE)
    chroma = librosa.feature.chroma_stft(y=y, sr=RATE)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=RATE)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=RATE)
    rmse = librosa.feature.rms(y=y)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=RATE)
    
    feature_vector = np.hstack([
        np.mean(mfccs, axis=1),
        np.mean(spectral_contrast, axis=1),
        np.mean(chroma, axis=1),
        np.mean(zero_crossing_rate),
        np.mean(spectral_centroid),
        np.mean(spectral_bandwidth),
        np.mean(rmse),
        np.mean(spectral_rolloff)
    ])
    
    return feature_vector

In [19]:
# Callback function to process each audio chunk
def callback(indata, frames, time, status):
    if status:
        print(f"Status: {status}")

    # Reshape and process the audio chunk
    audio_chunk = indata[:, 0]  # Use the first channel
    features = extract_single_audio_features(audio_chunk)

    # Reshape for scaling and scaling the features
    features = features.reshape(1, -1)
    scaled_features = scaler.transform(features)

    # Make a prediction
    prediction = model.predict(scaled_features)

    if prediction == 1:  # 1 indicates scream
        print("Scream detected!")

In [20]:
"""
# Start the microphone input stream
with sd.InputStream(callback=callback, channels=1, samplerate=RATE, blocksize=CHUNK_SIZE):
    print("Listening for screams...")
    while True:
        sd.sleep(int(CHUNK_DURATION * 1000))  # Wait for the chunk duration
"""

'\n# Start the microphone input stream\nwith sd.InputStream(callback=callback, channels=1, samplerate=RATE, blocksize=CHUNK_SIZE):\n    print("Listening for screams...")\n    while True:\n        sd.sleep(int(CHUNK_DURATION * 1000))  # Wait for the chunk duration\n'